Import dan load dataset CIFAR-10

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Memuat dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalisasi data (dari nilai 0-255 ke 0-1)
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encoding label
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Info dataset
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

Membangun model CNN

In [ ]:
# Membuat model CNN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer dengan 10 kelas
])

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Tampilkan ringkasan model
model.summary()

Melakukan training

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_test, y_test)
)

Menampilkan akurasi dan confusion matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix 
import seaborn as sns

# Fungsi untuk menampilkan plot akurasi, loss, dan confusion matrix
def plot_training_history_and_confusion_matrix(history, model, x_test, y_test):
    # Menampilkan plot Akurasi dan Loss
    plt.figure(figsize=(12, 6))
    
    # Plot Akurasi
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy during Training')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss during Training')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    # Menampilkan akurasi dan loss akhir
    print(f"Training Accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print(f"Training Loss: {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")
    
    # Prediksi dengan model
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)

    # Ubah y_test menjadi bentuk label kelas (bukan one-hot encoding)
    y_test_classes = np.argmax(y_test, axis=1)

    # Hitung confusion matrix
    cm = confusion_matrix(y_test_classes, predicted_classes)  # Gunakan y_test_classes

    # Plot confusion matrix menggunakan seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Prediksi')
    plt.ylabel('Kelas Asli')
    plt.title('Confusion Matrix CIFAR-10')
    plt.show()
    
# Label CIFAR-10
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Menampilkan plot dan confusion matrix
plot_training_history_and_confusion_matrix(history, model, x_test, y_test)

Simpan Model

In [ ]:
# Simpan model ke file .h5
model.save('cifar10_cnn_model.h5')
print("Model telah disimpan ke file cifar10_cnn_model.h5")

Load model dan test

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Muat model dari file
loaded_model = load_model('cifar10_cnn_model.h5')
print("Model berhasil dimuat.")

# Prediksi contoh data
sample_image = np.expand_dims(x_test[0], axis=0)  # Pilih satu gambar dari test set
prediction = loaded_model.predict(sample_image)
predicted_class = np.argmax(prediction)

# Daftar label CIFAR-10
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

print(f"Prediksi: {labels[predicted_class]}")